In [1]:
# Autor: Pablo Rocamora

import matplotlib.pyplot as plt
import neptune
import numpy as np
import random
import seaborn as sns
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from collections import defaultdict
from dataset_handlers import (FiberCupHandler, 
                              HCPHandler, 
                              TractoinfernoHandler, 
                              HCP_Without_CC_Handler)
from encoders import (
    GCNEncoder,
    ProjectionHead
)
from graph_transformer import GraphTransformerEncoder
from loss_functions import MultiTaskTripletLoss
from streamline_datasets import (
    MaxMinNormalization,
    StreamlineTestDataset,
    StreamlineTripletDataset,
    collate_test_ds,
    collate_triplet_ds,
    fill_tracts_ds
)
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from torchmetrics.classification import (
    MulticlassAUROC,
    MulticlassAccuracy,
    MulticlassConfusionMatrix,
    MulticlassF1Score
)
from tqdm import tqdm
import os
from gcn_encoder_model_v2 import SiameseGraphNetworkGCN_v2
import torch
import torch.nn as nn
from torch_geometric.nn import GCNConv,GraphConv, global_mean_pool, BatchNorm
from torch.nn import ModuleList
import torch.nn.functional as F


In [2]:

# Comando para lanzar tensorboard en el navegador local a través del puerto 8888 reenviado por ssh:
# tensorboard --logdir=runs/embedding_visualization --host 0.0.0.0 --port 8888


# Enable TensorFloat32 for better performance in matrix multiplication
torch.set_float32_matmul_precision('high')

# Seed setting function
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Función para guardar un checkpoint
def save_checkpoint(epoch, model, optimizer, loss, filename='checkpoint.pth'):
    checkpoint_dir = '/app/trained_models'
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss
    }
    torch.save(checkpoint, os.path.join(checkpoint_dir, filename))




# log = True
# if log:
#     run = neptune.init_run(
#         project="pablorocamora/tfm-tractography",
#         api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI1ODA0YzA2NS04MjczLTQyNzItOGE5Mi05ZmI5YjZkMmY3MDcifQ==",
#     )

    
    

# Configuration class
class CFG:
    def __init__(self):
        self.seed = 42
        self.max_epochs = 4
        self.batch_size = 1024
        self.learning_rate = 1e-4
        self.max_batches_per_subject = 150# Un buen valor es 500
        self.optimizer = "AdamW"
        self.classification_weight = 1
        self.margin = 1.0
        self.encoder = "GCNEncoder_v2"
        self.embedding_projection_dim = 512
        self.dataset = "HCP_105"#"Tractoinferno"

        dataset_paths = {
            "HCP_105": ("/app/dataset/HCP_105", 72),
            "HCP_105_without_CC": ("/app/dataset/HCP_105", 71),
            "Tractoinferno": ("/app/dataset/Tractoinferno/tractoinferno_preprocessed_mni", 32),
            "FiberCup": ("/app/dataset/Fibercup", 7)
        }
        self.ds_path, self.n_classes = dataset_paths.get(self.dataset, (None, None))
    
    
# Initialize configuration
cfg = CFG()
# if log:
#     writer = SummaryWriter(log_dir=f"runs/{cfg.dataset}_embedding_visualization", filename_suffix=f"{time.time()}")
#     run["config"] = {
#         "dataset": cfg.dataset,
#         "seed": cfg.seed,
#         "max_epochs": cfg.max_epochs,
#         "batch_size": cfg.batch_size,
#         "learning_rate": cfg.learning_rate,
#         "max_batches_per_subject": cfg.max_batches_per_subject,
#         "n_classes": cfg.n_classes,
#         "embedding_projection_dim": cfg.embedding_projection_dim,
#         "classification_weight": cfg.classification_weight,
#         "margin": cfg.margin,
#         "encoder": cfg.encoder,
#         "optimizer": cfg.optimizer
#     }

# Set the seed
seed_everything(cfg.seed)



# Cargar las rutas de los sujetos de entrenamiento, validación y test
if cfg.dataset == "HCP_105":
    handler = HCPHandler(path = cfg.ds_path, scope = "trainset")
    train_data = handler.get_data()

    handler = HCPHandler(path = cfg.ds_path, scope = "validset")
    valid_data = handler.get_data()

    handler = HCPHandler(path = cfg.ds_path, scope = "testset")
    test_data = handler.get_data()

elif cfg.dataset == "HCP_105_without_CC":
    handler = HCP_Without_CC_Handler(path = cfg.ds_path, scope = "trainset")
    train_data = handler.get_data()

    handler = HCP_Without_CC_Handler(path = cfg.ds_path, scope = "validset")
    valid_data = handler.get_data()

    handler = HCP_Without_CC_Handler(path = cfg.ds_path, scope = "testset")
    test_data = handler.get_data()

elif cfg.dataset == "Tractoinferno":
    handler = TractoinfernoHandler(path = cfg.ds_path, scope = "trainset")
    train_data = handler.get_data()
    train_data = fill_tracts_ds(train_data)# Hacer que todos los sujetos tengan el mismo número de tractos 

    handler = TractoinfernoHandler(path = cfg.ds_path, scope = "validset")
    valid_data = handler.get_data()

    handler = TractoinfernoHandler(path = cfg.ds_path, scope = "testset")
    test_data = handler.get_data()

elif cfg.dataset == "FiberCup":
    handler = FiberCupHandler(path = cfg.ds_path, scope = "trainset")
    train_data = handler.get_data()

    handler = FiberCupHandler(path = cfg.ds_path, scope = "validset")
    valid_data = handler.get_data()

    handler = FiberCupHandler(path = cfg.ds_path, scope = "testset")
    test_data = handler.get_data()



{0: 'AF_left', 1: 'AF_right', 2: 'ATR_left', 3: 'ATR_right', 4: 'CA', 5: 'CC_1', 6: 'CC_2', 7: 'CC_3', 8: 'CC_4', 9: 'CC_5', 10: 'CC_6', 11: 'CC_7', 12: 'CC', 13: 'CG_left', 14: 'CG_right', 15: 'CST_left', 16: 'CST_right', 17: 'MLF_left', 18: 'MLF_right', 19: 'FPT_left', 20: 'FPT_right', 21: 'FX_left', 22: 'FX_right', 23: 'ICP_left', 24: 'ICP_right', 25: 'IFO_left', 26: 'IFO_right', 27: 'ILF_left', 28: 'ILF_right', 29: 'MCP', 30: 'OR_left', 31: 'OR_right', 32: 'POPT_left', 33: 'POPT_right', 34: 'SCP_left', 35: 'SCP_right', 36: 'SLF_I_left', 37: 'SLF_I_right', 38: 'SLF_II_left', 39: 'SLF_II_right', 40: 'SLF_III_left', 41: 'SLF_III_right', 42: 'STR_left', 43: 'STR_right', 44: 'UF_left', 45: 'UF_right', 46: 'T_PREF_left', 47: 'T_PREF_right', 48: 'T_PREM_left', 49: 'T_PREM_right', 50: 'T_PREC_left', 51: 'T_PREC_right', 52: 'T_POSTC_left', 53: 'T_POSTC_right', 54: 'T_PAR_left', 55: 'T_PAR_right', 56: 'T_OCC_left', 57: 'T_OCC_right', 58: 'ST_FO_left', 59: 'ST_FO_right', 60: 'ST_PREF_left', 6

In [3]:
# Modelo para cargar pesos y finetunear
#============================================================================
class GraphConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, dropout=0.0):
        super(GraphConvBlock, self).__init__()
        self.conv = GCNConv(in_channels, out_channels)
        self.bn = BatchNorm(out_channels)
        self.relu = nn.LeakyReLU()
        self.dropout = nn.Dropout(p=dropout) if dropout > 0 else None

    def forward(self, x, edge_index):
        x = self.conv(x, edge_index)
        x = self.bn(x)
        x = self.relu(x)
        if self.dropout:
            x = self.dropout(x)
        return x
    

class GCNEncoder(nn.Module):
    def __init__(self, in_channels, hidden_dim, out_channels, dropout, n_hidden_blocks):
        super(GCNEncoder, self).__init__()
        self.input_block = GraphConvBlock(in_channels, hidden_dim, dropout)
        self.hidden_blocks = ModuleList([GraphConvBlock(hidden_dim, hidden_dim, dropout) for _ in range(n_hidden_blocks - 1)])
        self.output_block = GraphConvBlock(hidden_dim, out_channels, dropout)
        self.bn = BatchNorm(out_channels)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = self.input_block(x, edge_index)
        for layer in self.hidden_blocks:
            x = layer(x, edge_index)
        x = self.output_block(x, edge_index)
        x = self.bn(x)

        return global_mean_pool(x, batch) # (batch_size, out_channels)


class ProjectionHead(nn.Module):
    """
    Proyección de las embeddings de texto a un espacio de dimensión reducida.
    """
    def __init__(
        self,
        embedding_dim,# Salida del modelo de lenguaje (768)
        projection_dim, # Dimensión de la proyección (256)
        # dropout=0.1
    ):
        super(ProjectionHead, self).__init__()
        self.projection = nn.Linear(embedding_dim, projection_dim)
        self.gelu = nn.GELU()
        self.fc = nn.Linear(projection_dim, projection_dim)
        # self.dropout = nn.Dropout(dropout)
        self.layer_norm = nn.LayerNorm(projection_dim)

    def forward(self, x):
        projected = self.projection(x)
        x = self.gelu(projected)
        x = self.fc(x)
        # x = self.dropout(x)
        x = x + projected
        x = self.layer_norm(x)
        return x
    

class SiameseContrastiveGraphNetwork(nn.Module):
    def __init__(self, encoder, projection_head):
        super(SiameseContrastiveGraphNetwork, self).__init__()
        self.encoder = encoder
        self.projection_head = projection_head

    def forward(self, graph):
        x_1 = self.encoder(graph)
        x_1 = self.projection_head(x_1)
        # x1_norm = F.normalize(x_1, p=2, dim=1)
        return x_1


model = SiameseContrastiveGraphNetwork(
    encoder = GCNEncoder(
        in_channels = 3, 
        hidden_dim = 128, 
        out_channels = 512, 
        dropout = 0.15, 
        n_hidden_blocks = 2
    ),

    projection_head = ProjectionHead(
        embedding_dim = 512, 
        projection_dim = 128
    )
).cuda()
model = torch.compile(model, dynamic=True)

# Cargar pesos preentrenados
checkpoint = torch.load('/app/trained_models/checkpoint_HCP_105_GCN_512_1_infonce_1723061405.9630373.pth')

model.load_state_dict(checkpoint['model_state_dict'])


# Eliminar la última capa de proyección
model = model.encoder
# Congelar los pesos del modelo
for param in model.parameters():
    param.requires_grad = False

# Añadir una capa de clasificación
class FinalModel(nn.Module):
    def __init__(self, base_model):
        super(FinalModel, self).__init__()
        self.base_model = base_model
        self.fc = nn.Linear(512, 512)
        self.classifier = nn.Linear(512, 71)

    def forward(self, graph):
        x = self.base_model(graph)
        x = self.fc(x)
        x = F.relu(x)
        x = self.classifier(x)
        return x

model = FinalModel(model).cuda()

model

RuntimeError: Error(s) in loading state_dict for OptimizedModule:
	size mismatch for _orig_mod.encoder.input_block.conv.bias: copying a param with shape torch.Size([64]) from checkpoint, the shape in current model is torch.Size([128]).
	size mismatch for _orig_mod.encoder.input_block.conv.lin.weight: copying a param with shape torch.Size([64, 3]) from checkpoint, the shape in current model is torch.Size([128, 3]).
	size mismatch for _orig_mod.encoder.input_block.bn.module.weight: copying a param with shape torch.Size([64]) from checkpoint, the shape in current model is torch.Size([128]).
	size mismatch for _orig_mod.encoder.input_block.bn.module.bias: copying a param with shape torch.Size([64]) from checkpoint, the shape in current model is torch.Size([128]).
	size mismatch for _orig_mod.encoder.input_block.bn.module.running_mean: copying a param with shape torch.Size([64]) from checkpoint, the shape in current model is torch.Size([128]).
	size mismatch for _orig_mod.encoder.input_bloc

In [4]:
# Comprobar que capas son entrenables
for name, param in model.named_parameters():
    print(name, param.requires_grad)

base_model.input_block.conv.bias False
base_model.input_block.conv.lin.weight False
base_model.input_block.bn.module.weight False
base_model.input_block.bn.module.bias False
base_model.hidden_blocks.0.conv.bias False
base_model.hidden_blocks.0.conv.lin.weight False
base_model.hidden_blocks.0.bn.module.weight False
base_model.hidden_blocks.0.bn.module.bias False
base_model.output_block.conv.bias False
base_model.output_block.conv.lin.weight False
base_model.output_block.bn.module.weight False
base_model.output_block.bn.module.bias False
base_model.bn.module.weight False
base_model.bn.module.bias False
fc.weight True
fc.bias True
classifier.weight True
classifier.bias True


In [5]:



# Definir loss function, optimizador y scheduler
# Cross entropy loss
criterion = nn.CrossEntropyLoss(
    weight = torch.tensor([1.0]*cfg.n_classes)
).cuda()


optimizer = torch.optim.AdamW(
    model.parameters(), 
    lr = cfg.learning_rate,
    weight_decay = 1e-4
)


scheduler = torch.optim.lr_scheduler.StepLR(
    optimizer, 
    step_size = 10, 
    gamma = 0.1
)




# Crear las métricas con torchmetrics
subj_accuracy_train = MulticlassAccuracy(num_classes = cfg.n_classes, average='macro').cuda()
subj_accuracy_val = MulticlassAccuracy(num_classes = cfg.n_classes, average='macro').cuda()

subj_f1_train = MulticlassF1Score(num_classes = cfg.n_classes, average='macro').cuda()
subj_f1_val = MulticlassF1Score(num_classes = cfg.n_classes, average='macro').cuda()

subj_auroc_train = MulticlassAUROC(num_classes = cfg.n_classes).cuda()
subj_auroc_val = MulticlassAUROC(num_classes = cfg.n_classes).cuda()

subj_confusion_matrix = MulticlassConfusionMatrix(num_classes = cfg.n_classes).cuda()




# Entrenar el modelo
for epoch in range(cfg.max_epochs):

    #shuffle train_data for each epoch
    random.shuffle(train_data)

    model.train()# Establecer el modelo en modo de entrenamiento

    for idx_suj, subject in enumerate(train_data):# Iterar sobre los sujetos de entrenamiento

        # Crear el dataset y el dataloader
        train_ds = StreamlineTripletDataset(
            subject, 
            handler, 
            transform = MaxMinNormalization(dataset=cfg.dataset)
        )
        
        train_dl = DataLoader(
            train_ds, 
            batch_size = cfg.batch_size, 
            shuffle = True, 
            num_workers = 1,              
            collate_fn = collate_triplet_ds
        )
        
        # Bucle de entrenamiento del modelo
        prog_bar = tqdm(
            iterable = train_dl, 
            total = cfg.max_batches_per_subject
        )

        for i, (graph_anch, _, _) in enumerate(prog_bar):

            # Enviar a la gpu
            graph_anch = graph_anch.to('cuda')
            target = graph_anch.y

            # Reiniciar los gradientes
            optimizer.zero_grad()

            # Forward pass
            preds = model(graph_anch)# Batch size necesario para calcular la media de los embeddings

            # Calcular la pérdida
            loss = criterion(preds, target)
            
            # Backward pass
            loss.backward()

            # Actualizar los pesos
            optimizer.step()

            # Concatenar las predicciones de los dos grafos
            # preds = torch.argmax(pred, dim=-1)
            #argmax(pred, dim=-1)

            # Concatenar las etiquetas de los dos grafos
            targets = target


            # calcular métricas de entrenamiento
            subj_accuracy_train.update(preds, targets)
            subj_f1_train.update(preds, targets)
            subj_auroc_train.update(preds, targets)


            # Mostar métricas de entrenamiento cada 100 batches
            if i % 25 == 0:
                print(f"[TRAIN] Epoch {epoch+1}/{cfg.max_epochs} - Subj {idx_suj} - Batch {i} - Loss {loss.item()} - Acc.: {subj_accuracy_train.compute().item():.4f}, F1: {subj_f1_train.compute().item():.4f}, AUROC: {subj_auroc_train.compute().item():.4f}")
                
                # if log:# Loggear las métricas de batch
                #     run["train/batch/loss"].log(loss.item())
                #     run["train/batch/acc"].log(subj_accuracy_train.compute().item())
                #     run["train/batch/f1"].log(subj_f1_train.compute().item())
                #     run["train/batch/auroc"].log(subj_auroc_train.compute().item())

            # Condicion de parada del bucle
            if i == cfg.max_batches_per_subject:

                # if log:# Loggear las métricas de subject
                #     run["train/subject/acc"].log(subj_accuracy_train.compute().item())
                #     run["train/subject/f1"].log(subj_f1_train.compute().item())
                #     run["train/subject/auroc"].log(subj_auroc_train.compute().item())
                
                # Reiniciar las métricas
                subj_accuracy_train.reset()
                subj_f1_train.reset()
                subj_auroc_train.reset()

                break
        
        # Actualizar el learning rate
        scheduler.step()

    # Save the model checkpoint
    checkpoint_name = f'checkpoint_{cfg.dataset}_{cfg.encoder}_{cfg.embedding_projection_dim}_{epoch}__finetuned.pth'
    save_checkpoint(epoch, model, optimizer, loss, filename=checkpoint_name)


  2%|▏         | 3/150 [00:00<00:27,  5.29it/s]

[TRAIN] Epoch 1/4 - Subj 0 - Batch 0 - Loss 4.264024257659912 - Acc.: 0.0061, F1: 0.0056, AUROC: 0.4925


 18%|█▊        | 27/150 [00:04<00:21,  5.86it/s]

[TRAIN] Epoch 1/4 - Subj 0 - Batch 25 - Loss 3.813040018081665 - Acc.: 0.2299, F1: 0.1890, AUROC: 0.8526


 35%|███▍      | 52/150 [00:08<00:15,  6.33it/s]

...

[TRAIN] Epoch 1/4 - Subj 43 - Batch 100 - Loss 0.2744048237800598 - Acc.: 0.8943, F1: 0.8929, AUROC: 0.9988


 85%|████████▍ | 127/150 [00:21<00:03,  5.78it/s]

[TRAIN] Epoch 1/4 - Subj 43 - Batch 125 - Loss 0.2433452606201172 - Acc.: 0.8947, F1: 0.8933, AUROC: 0.9988


 93%|█████████▎| 139/150 [00:23<00:01,  5.83it/s]


KeyboardInterrupt: 

In [ ]:


#     # Fase de validación del modelo
#     model.eval()
    
#     for idx_val, subject in enumerate(valid_data):
#         val_ds = StreamlineTestDataset(subject, handler, 
#                                         transform = MaxMinNormalization(dataset = cfg.dataset))
        
#         val_dl = DataLoader(val_ds, batch_size = cfg.batch_size , 
#                               shuffle = False, num_workers = 1,
#                               collate_fn = collate_test_ds)
        
#         # Diccionario para guardar los embeddings de los grafos por clase cuyas claves son las etiquetas 0 - 71
#         # Diccionario para guardar los embeddings por clase
#         embeddings_list_by_class = defaultdict(list)
#         max_embeddings_per_class = 100
        

#         with torch.no_grad():
#             for i, graph in enumerate(val_dl):
                
#                 # Enviar a la gpu
#                 graph = graph.to('cuda')
#                 target = graph.y

#                 # Forward pass
#                 embedding, pred = model(graph)# Batch size necesario para calcular la media de los embeddings

#                 # Calcular métricas de validación
#                 subj_accuracy_val.update(pred, target)
#                 subj_f1_val.update(pred, target)
#                 subj_auroc_val.update(pred, target)
#                 subj_confusion_matrix.update(pred, target)

#                 # Guardar los embeddings y etiquetas
#                 if idx_val == 0:

#                     embedding = embedding.cpu().numpy()
#                     target = graph.y.cpu().numpy()

#                     # Guardar embeddings y etiquetas en el diccionario por clase
#                     for emb, label in zip(embedding, target):
#                         if len(embeddings_list_by_class[label]) < max_embeddings_per_class:
#                             embeddings_list_by_class[label].append(emb)
                    

#                 if log:# Loggear las métricas de batch
#                     run["val/batch/acc"].log(subj_accuracy_val.compute().item())
#                     run["val/batch/f1"].log(subj_f1_val.compute().item())
#                     run["val/batch/auroc"].log(subj_auroc_val.compute().item())

#                 if i % 25 == 0:
#                     print(f"[VAL] Epoch {epoch+1}/{cfg.max_epochs} - Subj {idx_val} - Batch {i} - Acc.: {subj_accuracy_val.compute().item():.4f}, F1: {subj_f1_val.compute().item():.4f}, AUROC: {subj_auroc_val.compute().item():.4f}")

#             # Para el idx_val = 1, guardar los embeddings de los grafos con sus etiquetas para visualizarlos en TensorBoard Projector
#             if idx_val == 0:

#                 # Preparar los datos para TensorBoard
#                 all_embeddings = []
#                 all_labels = []

#                 for label, embeddings in embeddings_list_by_class.items():
#                     all_embeddings.extend(embeddings)
#                     # Obtener la etiqueta textual de la clase a través del handler
#                     label = handler.get_tract_from_label(label)
#                     print(label)
#                     all_labels.extend([label] * len(embeddings))

#                 # Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /opt/conda/conda-bld/pytorch_1704987394225/work/torch/csrc/utils/tensor_new.cpp:275.)
#                 # Convertir listas a numpy.ndarray
#                 all_embeddings = np.array(all_embeddings)
#                 all_labels = np.array(all_labels)


#                 # Convertir a tensores
#                 # Convertir a tensores
#                 all_embeddings = torch.tensor(all_embeddings)
#                 all_labels = all_labels.tolist()  # TensorBoard necesita etiquetas como lista de strings


#                 # Guardar los embeddings y etiquetas en TensorBoard
#                 writer.add_embedding(
#                     all_embeddings, 
#                     metadata = all_labels, 
#                     global_step = epoch,
#                     tag = f"{cfg.dataset}_{cfg.encoder}_{cfg.embedding_projection_dim}_{time.time()}"
#                 ) 

#             if log:# Loggear las métricas de subject
#                 run["val/subject/acc"].log(subj_accuracy_val.compute().item())
#                 run["val/subject/f1"].log(subj_f1_val.compute().item())
#                 run["val/subject/auroc"].log(subj_auroc_val.compute().item())

#                 cm = subj_confusion_matrix.compute()
#                 # Convertir la matriz de confusión a numpy
#                 cm = cm.cpu().numpy()

#                 # Visualiza la matriz de confusión y guárdala como imagen
#                 plt.figure(figsize=(35, 35))
#                 sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')

#                 text_labels = [handler.get_tract_from_label(i) for i in range(cfg.n_classes)]
#                 plt.xticks(ticks = range(cfg.n_classes), labels = text_labels, rotation = 90)
#                 plt.yticks(ticks = range(cfg.n_classes), labels = text_labels, rotation = 0)
#                 plt.xlabel('Predicted Labels')
#                 plt.ylabel('True Labels')
#                 plt.title(f'Confusion Matrix Subj {idx_val}')
#                 plt.tight_layout()

#                 # Guarda la imagen
#                 img_path = f'/app/confusion_matrix_imgs/confusion_matrix_val_suj{idx_val}.png'
#                 plt.savefig(img_path)
#                 plt.close()

#                 # Sube la imagen a Neptune
#                 run["confusion_matrix_fig"].upload(img_path)
            
#             subj_accuracy_val.reset()
#             subj_f1_val.reset()
#             subj_auroc_val.reset()
#             subj_confusion_matrix.reset()

#         if idx_val == 6:
#             break

# # Cerrar el SummaryWriter
# writer.close()

                   


Ejemplo de configuracion del clasificador preentrenado 

In [7]:
# Autor: Pablo Rocamora

import torch
import torch.nn as nn
import torch.nn.functional as F
from encoders import (
    ClassifierHead,
    GATEncoder,
    GCNEncoder,
    ProjectionHead,
    SiameseGraphNetwork
)

from torch_geometric.nn import GCNConv, global_mean_pool, BatchNorm

from encoders import (
    ClassifierHead,
    GCNEncoder,
    ProjectionHead,
)


class GraphConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, dropout=0.0):
        super(GraphConvBlock, self).__init__()
        self.conv = GCNConv(in_channels, out_channels)
        self.bn = BatchNorm(out_channels)
        self.relu = nn.LeakyReLU()
        self.dropout = nn.Dropout(p=dropout) if dropout > 0 else None

    def forward(self, x, edge_index):
        x = self.conv(x, edge_index)
        x = self.relu(x)
        x = self.bn(x)
        if self.training:  # Aplicación de Dropout solo durante el entrenamiento
            x = self.dropout(x)
        
        return x
    

class GCNEncoder(nn.Module):
    def __init__(self, in_channels, hidden_dim, out_channels, dropout, n_hidden_blocks):
        super(GCNEncoder, self).__init__()
        self.input_block = GraphConvBlock(in_channels, hidden_dim, dropout)
        self.hidden_blocks = nn.ModuleList([GraphConvBlock(hidden_dim, hidden_dim, dropout) for _ in range(n_hidden_blocks - 1)])
        self.output_block = GraphConvBlock(hidden_dim, out_channels, dropout)
        # self.bn = BatchNorm(out_channels)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = self.input_block(x, edge_index)
        for layer in self.hidden_blocks:
            x = layer(x, edge_index)
        x = self.output_block(x, edge_index)
        # x = self.bn(x)

        return global_mean_pool(x, batch) # (batch_size, out_channels)


class ProjectionHead(nn.Module):
    """
    Proyección de las embeddings de texto a un espacio de dimensión reducida.
    """
    def __init__(
        self,
        embedding_dim,# Salida del modelo de lenguaje (768)
        projection_dim, # Dimensión de la proyección (256)
        # dropout=0.1
    ):
        super(ProjectionHead, self).__init__()
        self.projection = nn.Linear(embedding_dim, projection_dim)
        self.gelu = nn.GELU()
        self.fc = nn.Linear(projection_dim, projection_dim)
        # self.dropout = nn.Dropout(dropout)
        self.layer_norm = nn.LayerNorm(projection_dim)

    def forward(self, x):
        projected = self.projection(x)
        x = self.gelu(projected)
        x = self.fc(x)
        # x = self.dropout(x)
        x = x + projected
        x = self.layer_norm(x)
        return x
    

class SiameseContrastiveGraphNetwork(nn.Module):
    def __init__(self, encoder, projection_head):
        super(SiameseContrastiveGraphNetwork, self).__init__()
        self.encoder = encoder
        self.projection_head = projection_head

    def forward(self, graph):
        x_1 = self.encoder(graph)
        x_1 = self.projection_head(x_1)
        return x_1

# ========================================================================


In [14]:

model = SiameseContrastiveGraphNetwork(
    encoder = GCNEncoder(
        in_channels = 3, 
        hidden_dim = 64, 
        out_channels = 128, 
        dropout = 0.5, 
        n_hidden_blocks = 4
    ),

    projection_head = ProjectionHead(
        embedding_dim = 128, 
        projection_dim = 64
    )
)

model = torch.compile(model, dynamic=True)

# Cargar pesos preentrenados
checkpoint = torch.load('/app/trained_models/checkpoint_HCP_105_GCN_512_5_infonce_1723148869.1645732.pth')

model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [15]:
model

OptimizedModule(
  (_orig_mod): SiameseContrastiveGraphNetwork(
    (encoder): GCNEncoder(
      (input_block): GraphConvBlock(
        (conv): GCNConv(3, 64)
        (bn): BatchNorm(64)
        (relu): LeakyReLU(negative_slope=0.01)
        (dropout): Dropout(p=0.5, inplace=False)
      )
      (hidden_blocks): ModuleList(
        (0-2): 3 x GraphConvBlock(
          (conv): GCNConv(64, 64)
          (bn): BatchNorm(64)
          (relu): LeakyReLU(negative_slope=0.01)
          (dropout): Dropout(p=0.5, inplace=False)
        )
      )
      (output_block): GraphConvBlock(
        (conv): GCNConv(64, 128)
        (bn): BatchNorm(128)
        (relu): LeakyReLU(negative_slope=0.01)
        (dropout): Dropout(p=0.5, inplace=False)
      )
    )
    (projection_head): ProjectionHead(
      (projection): Linear(in_features=128, out_features=64, bias=True)
      (gelu): GELU(approximate='none')
      (fc): Linear(in_features=64, out_features=64, bias=True)
      (layer_norm): LayerNorm((64,

In [17]:
# Delete dropout layers and freeze the weights
model.encoder.input_block.dropout = None
model.encoder.hidden_blocks[0].dropout = None
model.encoder.hidden_blocks[1].dropout = None
model.encoder.hidden_blocks[2].dropout = None

model.encoder.output_block.dropout = None

# Delete projection head
model = model.encoder

for param in model.parameters():
    param.requires_grad = False

model    

GCNEncoder(
  (input_block): GraphConvBlock(
    (conv): GCNConv(3, 64)
    (bn): BatchNorm(64)
    (relu): LeakyReLU(negative_slope=0.01)
    (dropout): None
  )
  (hidden_blocks): ModuleList(
    (0-2): 3 x GraphConvBlock(
      (conv): GCNConv(64, 64)
      (bn): BatchNorm(64)
      (relu): LeakyReLU(negative_slope=0.01)
      (dropout): None
    )
  )
  (output_block): GraphConvBlock(
    (conv): GCNConv(64, 128)
    (bn): BatchNorm(128)
    (relu): LeakyReLU(negative_slope=0.01)
    (dropout): None
  )
)

In [19]:

class GraphClassifier(nn.Module):
    def __init__(self, encoder, n_classes):
        super(GraphClassifier, self).__init__()
        self.encoder = encoder

        self.classifier = ClassifierHead(
            projection_dim = 128, 
            n_classes = n_classes
        )   

    def forward(self, graph):
        x = self.encoder(graph)
        x = self.classifier(x)
        return x

model = GraphClassifier(model, 72)

model

GraphClassifier(
  (encoder): GCNEncoder(
    (input_block): GraphConvBlock(
      (conv): GCNConv(3, 64)
      (bn): BatchNorm(64)
      (relu): LeakyReLU(negative_slope=0.01)
      (dropout): None
    )
    (hidden_blocks): ModuleList(
      (0-2): 3 x GraphConvBlock(
        (conv): GCNConv(64, 64)
        (bn): BatchNorm(64)
        (relu): LeakyReLU(negative_slope=0.01)
        (dropout): None
      )
    )
    (output_block): GraphConvBlock(
      (conv): GCNConv(64, 128)
      (bn): BatchNorm(128)
      (relu): LeakyReLU(negative_slope=0.01)
      (dropout): None
    )
  )
  (classifier): ClassifierHead(
    (fc): Linear(in_features=128, out_features=72, bias=True)
    (softmax): LogSoftmax(dim=1)
  )
)